Based Arshia Azfal's code: Creat and format the train and test datasets for the Vision Transformer model
Window the training and test data, into 3 second windows, maintaining a single label accross the window.

In [84]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [85]:
import numpy as np
import os
import pickle
from tqdm import tqdm as tqdm

In [86]:
Train_data = np.load('/content/drive/MyDrive/Colab Notebooks/INL/processed_data/Train_data.npy', allow_pickle = True)
train_channels = np.load('/content/drive/MyDrive/Colab Notebooks/INL/processed_data/Ch_train.npy', allow_pickle = True)
Test_data = np.load('/content/drive/MyDrive/Colab Notebooks/INL/processed_data/Test_data.npy', allow_pickle = True)
test_channels = np.load('/content/drive/MyDrive/Colab Notebooks/INL/processed_data/Ch_test.npy', allow_pickle = True)

In [87]:
Train_data.shape

(192,)

In [88]:
Train_data[32].shape

(56, 162663)

In [89]:
train_channels[0]

['add_ch_0',
 'add_ch_1',
 'add_ch_2',
 'add_ch_3',
 'add_ch_4',
 'add_ch_5',
 'add_ch_6',
 'add_ch_7',
 'add_ch_8',
 'is_updating',
 'states',
 'state__idle',
 'state__shoulder__flexion',
 'state__shoulder__extension',
 'state__shoulder__adduction',
 'state__shoulder__abduction',
 'state__shoulder__internal_rotation',
 'state__shoulder__external_rotation',
 'state__elbow__flexion',
 'state__elbow__extension',
 'state__wrist__pronation',
 'state__wrist__supination',
 'state__hand__open',
 'state__hand__close',
 'ecog_0',
 'ecog_1',
 'ecog_2',
 'ecog_3',
 'ecog_4',
 'ecog_5',
 'ecog_6',
 'ecog_7',
 'ecog_8',
 'ecog_9',
 'ecog_10',
 'ecog_11',
 'ecog_12',
 'ecog_13',
 'ecog_14',
 'ecog_15',
 'ecog_16',
 'ecog_17',
 'ecog_18',
 'ecog_19',
 'ecog_20',
 'ecog_21',
 'ecog_22',
 'ecog_23',
 'ecog_24',
 'ecog_25',
 'ecog_26',
 'ecog_27',
 'ecog_28',
 'ecog_29',
 'ecog_30',
 'ecog_31']

In [90]:
# State Labels
label_names = train_channels[0][11:24]
idx2label = {index: value for index, value in enumerate(label_names)}
label2idx = {value: index for index, value in enumerate(label_names)}
idx2label

{0: 'state__idle',
 1: 'state__shoulder__flexion',
 2: 'state__shoulder__extension',
 3: 'state__shoulder__adduction',
 4: 'state__shoulder__abduction',
 5: 'state__shoulder__internal_rotation',
 6: 'state__shoulder__external_rotation',
 7: 'state__elbow__flexion',
 8: 'state__elbow__extension',
 9: 'state__wrist__pronation',
 10: 'state__wrist__supination',
 11: 'state__hand__open',
 12: 'state__hand__close'}

In [91]:
fs = 590 #Hz

X_train = []
y_train = []
X_test = []
y_test = []

for data in Train_data:

    if data.shape[0] == 56 and data.shape[1] != 0: # Only keep shape of 56 channels for simplicity and coherence
        y = data[11:24,:] #13 state channels
        x = data[-31:,:] #31 ecog channels
    else:
      continue

    assert x.shape[1] == y.shape[1] , 'Shape of the labels and ecog data does not match'

    for t in range(0,x.shape[1]-3*fs-1,fs):
        X_train.append( x[:,t:t+3*fs] )
        y_train.append( y[:,t:t+3*fs] )

for data in Test_data:

    if data.shape[0] == 56 and data.shape[1] != 0:
        y = data[11:24,:]
        x = data[-31:,:]
    else:
      continue

    assert x.shape[1] == y.shape[1] , 'Shape of the labels and ecog data does not match'

    for t in range(0,x.shape[1]-3*fs-1,fs):
        X_test.append(x[:,t:t+3*fs])
        y_test.append(y[:,t:t+3*fs])

In [92]:
X_train = np.array(X_train)
Y = np.array(y_train)
X_test = np.array(X_test)
Y_test = np.array(y_test)

In [93]:
X_train.shape

(21760, 31, 1770)

In [94]:
Y.shape

(21760, 13, 1770)

In [95]:
label = np.argmax(Y , axis=1) # Only the first occurence is kept, importance in order of labeling
print(f"label.shape: {label.shape}")
label_test = np.argmax(Y_test , axis=1) # Only the first occurence is kept, importance in order of labeling
print(f"label_test.shape: {label_test.shape}")

label.shape: (21760, 1770)
label_test.shape: (4201, 1770)


In [97]:
unique_values, counts = np.unique(label, return_counts=True)
idx_counts = dict(zip(unique_values, counts))
idx_counts

{0: 21678429,
 1: 3043928,
 7: 402380,
 8: 5700580,
 9: 2293684,
 10: 368514,
 11: 2975252,
 12: 2052433}

In [127]:
# Verify test has the same states represented
unique_values, counts = np.unique(label_test, return_counts=True)
idx_test_counts = dict(zip(unique_values, counts))
assert idx_test_counts.keys() == idx_counts.keys()
idx_test_counts

{0: 4144396,
 1: 217533,
 7: 155524,
 8: 736969,
 9: 716496,
 10: 33453,
 11: 870191,
 12: 561208}

In [98]:
label_counts = {idx2label[key]: value for key, value in idx_counts.items()}
label_counts

{'state__idle': 21678429,
 'state__shoulder__flexion': 3043928,
 'state__elbow__flexion': 402380,
 'state__elbow__extension': 5700580,
 'state__wrist__pronation': 2293684,
 'state__wrist__supination': 368514,
 'state__hand__open': 2975252,
 'state__hand__close': 2052433}

In [99]:
label_counts = {idx2label[key]: value for key, value in idx_counts.items()}
label_counts

for value in label_counts.keys():
  print(f"{label2idx[value]}, {value}: {label_counts[value]}")
#Not every label is represented in the training data

0, state__idle: 21678429
1, state__shoulder__flexion: 3043928
7, state__elbow__flexion: 402380
8, state__elbow__extension: 5700580
9, state__wrist__pronation: 2293684
10, state__wrist__supination: 368514
11, state__hand__open: 2975252
12, state__hand__close: 2052433


In [ ]:
label[label>=8] = label[label>=8]-6 # remap
label[label>=8] = label[label>=8]-6 # remap
num_cls_train = np.max(label) # number of classes

label_test[label_test>=8] = label_test[label_test>=8]-6 # remap
label_test[label_test>=8] = label_test[label_test>=8]-6 # remap
num_cls_test= np.max(label_test) # number of classes

In [73]:
# new labels after remapping
for value in label_counts.keys():
  print(f"{label2idx[value] - 6 if label2idx[value] >= 8 else label2idx[value]}, {value}: {label_counts[value]}")

0, state__idle: 21678429
1, state__shoulder__flexion: 3043928
7, state__elbow__flexion: 402380
2, state__elbow__extension: 5700580
3, state__wrist__pronation: 2293684
4, state__wrist__supination: 368514
5, state__hand__open: 2975252
6, state__hand__close: 2052433


In [117]:
# Sort and create new label dictionaries for the represented states in the dataset
new_label2idx = {key: label2idx[key] - 6 if label2idx[key] >= 8 else label2idx[key] for key in label_counts.keys()}
new_idx2label = dict(sorted({value: key for key, value in new_label2idx.items()}.items()))
new_label2idx = {value: key for key, value in new_idx2label.items()}
new_label2idx

{'state__idle': 0,
 'state__shoulder__flexion': 1,
 'state__elbow__extension': 2,
 'state__wrist__pronation': 3,
 'state__wrist__supination': 4,
 'state__hand__open': 5,
 'state__hand__close': 6,
 'state__elbow__flexion': 7}

In [ ]:
X_train , label = X_train[np.min(label,1) == np.max(label,1),:,:] ,  label[np.min(label,1) == np.max(label,1)] #ensure label is the same during the time window

X_test , label_test = X_test[np.min(label_test,1) == np.max(label_test,1),:,:] ,  label_test[np.min(label_test,1) == np.max(label_test,1)] #ensure label is the same during the time window

In [ ]:
label = np.mean(label,1) #Keep only the label of the time window
label.shape

label_test = np.mean(label_test,1) #Keep only the label of the time window
label_test.shape

(2237,)

In [ ]:
label.shape

(11730,)

In [ ]:
np.save('X_train.npy', X_train)
np.save('label.npy', label)

np.save('X_test.npy', X_test)
np.save('label_test.npy', label_test)